# [Module 1.3] 체크 포인트를 생성을 통한 스팟 인스턴스 훈련

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 체크포인트를 사용하는 방법
- 기본 환경 세팅
- 데이터 세트를 S3에 업로드
- 체크 포인트를 사용한 훈련 시니라오
    - 첫 번째 훈련 잡 실행
    - 두 번째 훈련 잡 실행
- 훈련 잡 로그 분석
- 모델 아티펙트 저장

---

## 세이지 메이커에서 체크포인트를 사용하는 방법

개발자 가이드 --> [체코 포인트 사용하기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-checkpoints.html)

![checkpoint_how.png](img/checkpoint_how.png)

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sagemaker
import uuid

sagemaker_session = sagemaker.Session()
print('SageMaker version: ' + sagemaker.__version__)

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-cnn-cifar10'

role = sagemaker.get_execution_role()


SageMaker version: 2.45.0


### 체크 포인트 파일 저장 경로
- S3에 체크포인트 경로를 지정합니다.

In [3]:
checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = 's3://{}/checkpoint-{}'.format(bucket, checkpoint_suffix)

print('Checkpointing Path: {}'.format(checkpoint_s3_path))

Checkpointing Path: s3://sagemaker-ap-northeast-2-057716757052/checkpoint-a7316680


#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [4]:
import os
import subprocess


try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
    else:
        instance_type = "local"        
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


### 데이터 세트를 S3에 업로드


In [5]:
inputs = sagemaker_session.upload_data(path="../data", bucket=bucket, key_prefix="data/cifar10")
print("s3 inputs: ", inputs)

s3 inputs:  s3://sagemaker-ap-northeast-2-057716757052/data/cifar10


## 체크포인트를 이용한 훈련 시나리오
총 훈련 작업은 10개의 epoch 까지를 실행을 합니다. 아래와 같이 두개의 훈련 잡을 통해서 합니다.
- 첫번째의 훈련잡은 5 epoch 까지만을 실행 합니다.
    - 매번의 epoch 마다 checkpoint 파일을 S3의  checkpoint_s3_uri 에 저장합니다.
    
    
```python
def _save_checkpoint(model, optimizer, epoch, loss, args):
    print("epoch: {} - loss: {}".format(epoch+1, loss))
    checkpointing_path = args.checkpoint_path + '/checkpoint.pth'
    print("Saving the Checkpoint: {}".format(checkpointing_path))
    torch.save({
        'epoch': epoch+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, checkpointing_path)


```
- 두번째의 훈련잡은 6 epoch 부터 10 epoch 까지 실행합니다.
    - 훈련이 시작시에  checkpoint_s3_uri 에서 마지막 훈련 결과(가중치)를 가져와서 모델에 로딩한 후에 시작 합니다.

### 스팟 인스턴스 훈련 시나리오
- 스팟 인스턴스로 훈련을 하다가 이 리소스가 다른 유저에게 빼앗기면, 훈련이 중단되고 스팟 인스턴스가 다시 사용가능할때에, checkpoint_s3_uri 에서 마지막 저장된 체크포인트를 가져와서 다시 훈련을 재개 합니다. 
- 상세 사항은 개발자 가이드를 보세요. --> [관리형 스팟 교육](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-managed-spot-training.html)

#### 체크포인트를 S3에 성공적으로 복사하려면 debugger_hook_config 매개 변수를 False로 설정해야 합니다. 



### 첫 번째 훈련 잡을 실행
- 스팟 인스턴스에 필요한 인자를 설정 합니다.
- 5 epochs 까지를 훈련 합니다.

In [6]:
use_spot_instances = True
max_run=600
max_wait = 1200 if use_spot_instances else None

In [7]:
hyperparameters = {'epochs': 5}

from sagemaker.pytorch import PyTorch
spot_estimator = PyTorch(
                            entry_point='cifar10.py',
                            source_dir='source',                                                            
                            role=role,
                            framework_version='1.6.0',
                            py_version='py3',
                            instance_count=1,
                            instance_type='ml.p3.2xlarge',
                            base_job_name='cifar10-pytorch-spot-1',
                            hyperparameters=hyperparameters,
                            checkpoint_s3_uri=checkpoint_s3_path,
                            debugger_hook_config=False,
                            use_spot_instances=use_spot_instances,
                            max_run=max_run,
                            max_wait=max_wait)

spot_estimator.fit(inputs, wait=False)

In [ ]:
spot_estimator.logs()

2021-07-29 03:59:33 Starting - Starting the training job...
2021-07-29 03:59:34 Starting - Launching requested ML instancesProfilerReport-1627531172: InProgress
......
2021-07-29 04:00:58 Starting - Preparing the instances for training.....

### 세이지 메이커 콘솔에서 체크포인트 확인
- 이제 SageMaker 콘솔에서 트레이닝 작업에서 체크포인트 구성을 직접 볼 수 있습니다.
- SageMaker 콘솔에 로그인하고 최신 교육 작업을 선택한 다음 체크포인트 구성 섹션으로 스크롤합니다.
- S3 출력 경로 링크를 선택하면 체크포인팅 데이터가 저장된 S3 버킷으로 연결됩니다.
- 거기에 하나의 파일 (checkpoint.pth) 이 있음을 알 수 있습니다.

![checkpoint_console-1.png](img/checkpoint_console.png)

### 두 번째 훈련 잡을 실행
- 이전 체크포인트 이후 부터 6 epochs ~ 10 epochs 까지를 훈련 합니다.
- 훈련 시작시에 다음의 단계가 진행 됩니다.
    - 체크포인트 s3 위치에서 체크포인트 데이터를 확인
    - 체크 포인트가 파일이 있을 경우 훈련 도커 컨테이너의 `/ opt/ml/체크포인트'에 복사됩니다.
- 아래의 체크 포인트 로딩하는 함수를 참조 하세요.


```python
def _load_checkpoint(model, optimizer, args):
    print("--------------------------------------------")
    print("Checkpoint file found!")
    print("Loading Checkpoint From: {}".format(args.checkpoint_path + '/checkpoint.pth'))
    checkpoint = torch.load(args.checkpoint_path + '/checkpoint.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch_number = checkpoint['epoch']
    loss = checkpoint['loss']
    print("Checkpoint File Loaded - epoch_number: {} - loss: {}".format(epoch_number, loss))
    print('Resuming training from epoch: {}'.format(epoch_number+1))
    print("--------------------------------------------")
    return model, optimizer, epoch_number
```



In [ ]:
hyperparameters = {'epochs': 10}


spot_estimator = PyTorch(
                            entry_point='cifar10.py',
                            source_dir='source',                                                            
                            role=role,
                            framework_version='1.6.0',
                            py_version='py3',
                            instance_count=1,
                            instance_type='ml.p3.2xlarge',
                            base_job_name='cifar10-pytorch-spot-2',
                            hyperparameters=hyperparameters,
                            checkpoint_s3_uri=checkpoint_s3_path,
                            debugger_hook_config=False,
                            use_spot_instances=use_spot_instances,
                            max_run=max_run,
                            max_wait=max_wait)

spot_estimator.fit(inputs, wait=False)

In [ ]:
spot_estimator.logs()

### 훈련 잡 로그 분석

훈련 잡 로그를 분석하면 훈련 잡 이 6번째 epoch 부터 시작된다는 것을 알 수 있습니다.

`_load_체크포인트` 함수의 출력을 볼 수 있습니다:

```
--------------------------------------------
Checkpoint file found!
Loading Checkpoint From: /opt/ml/checkpoints/checkpoint.pth
Checkpoint File Loaded - epoch_number: 5 - loss: 0.8455273509025574
Resuming training from epoch: 6
--------------------------------------------
```

훈련이 완료 된 후에 S3 의 체크포인트 파일이 업데이트가 됩니다.
```python
checkpoint.pth
```


## 모델 아티펙트 저장
- 아티펙트를 저장하여 추론에 사용합니다.

In [ ]:
spot_artifact_path = spot_estimator.model_data
print("spot_artifact_path: ", spot_artifact_path)

%store spot_artifact_path